In [11]:
import math
import numpy as np
import pandas as pd

from functools import lru_cache

In [12]:
def within_radius_mask(radius: float) -> np.ndarray:

    @lru_cache(maxsize=128)
    def within_radius(a, b, draws=1000000):
        '''
        This function returns the probability that the distance between a random point in the raster 0, 0 and a random
        point in the raster a, b is smaller than a given radius r.
        The units of a, b, and r are the resolution of the population raster (currently 1 km)
        :param r: cutoff radius
        :param a: distance of the rasters in one direction
        :param b: distnace of the rasters in the other direction
        :return: probability
        '''
        count = 0
        for _ in range(draws):
            x0, y0 = -0.5 + np.random.random(), -0.5 + np.random.random()
            x1, y1 = a - 0.5 + np.random.random(), b - 0.5 + np.random.random()
            distance = math.sqrt((x1-x0)**2 + (y1-y0)**2)
            if distance < radius:
                count += 1
        return count/draws

    ceil = int(np.ceil(radius))
    rng = range(-ceil, ceil+1)
    df = pd.DataFrame(index=rng, columns=rng, dtype=np.float64)
    for x in rng:
        for y in rng:
            a, b = sorted([abs(x), abs(y)])
            df.loc[x, y] = within_radius(a, b)

    return df.round(2).values

In [13]:
within_radius_mask(1)

array([[0.12, 0.42, 0.12],
       [0.42, 0.98, 0.42],
       [0.12, 0.42, 0.12]])